## 01_ETL

In [1]:
# importar librerias necesarias
import pandas as pd
import geopandas as gpd

##### Importar

In [2]:
# sig data
sig_provincias = gpd.read_file("data/sig/provincia.shp")
sig_localidades = gpd.read_file("data/sig/localidad_bahra.shp")

Cannot find header.dxf (GDAL_DATA is not defined)


In [3]:
# informacion a nivel nacional
velocidad_nac_por_año = pd.read_csv("data/unprocessed/historico_velocidad_internet.csv")
ingresos_nac = pd.read_csv("data/unprocessed/Internet_Ingresos.csv")
penetracion_nac = pd.read_csv("data/unprocessed/Internet_Penetracion (1).csv")
baf_nac = pd.read_csv("data/unprocessed/Internet_BAF.csv")
accesos_nac_por_tecnologia = pd.read_csv("data/unprocessed/Internet_Accesos-por-tecnologia.csv")
accesos_nac_por_velocidad = pd.read_csv("data/unprocessed/Internet_Accesos-por-velocidad nacionales transformado.csv")

In [4]:
# informacion a nivel provincial
accesos_prov_por_tecnologia = pd.read_csv("data/unprocessed/Internet_Accesos-por-tecnologia (1).csv")
accesos_prov_por_velocidad = pd.read_csv("data/unprocessed/AccesosaInternetfijoporvelocidadbajadayprovincia_transformado.csv")
penetracion_prov = pd.read_csv("data/unprocessed/Internet_Penetracion.csv")
baf_prov = pd.read_csv("data/unprocessed/Internet_BAF (1).csv")

In [5]:
# informacion a nivel local
accesos_loc_por_tecnologia = pd.read_csv("data/unprocessed/AccesosaInternetfijoportecnologiaylocalidad_2791751698918580638.csv")
accesos_loc_por_velocidad = pd.read_csv("data/unprocessed/AccesosaInternetfijoporvelocidaddebajadaylocalidad_transformado.csv")
localidades_con_conectividad = pd.read_csv("data/unprocessed/Listadodelocalidadesconconectividadainternet_2750221698918588798.csv")

##### Transformar

In [6]:
# sig data
sig_provincias.drop(columns=["gid", "entidad", "fna", "gna", "in1", "fdc", "sag"], inplace=True)
sig_provincias.rename(columns={"nam": "provincia"}, inplace=True)
sig_localidades.rename(columns={"nom_agl": "localidad", "nom_depto": "departamento", "nom_pcia": "provincia"}, inplace=True) 
sig_localidades.drop(columns=["gid", "cod_pcia", "cod_depto", "cod_ase", "fna", "cod_aglo", "nom_aglo", "cod_agl", "lat_gd", "long_gd", "lat_gs", "long_gs", "fdc", "tipo_asent"], inplace=True)

In [7]:
# informacion a nivel nacional
velocidad_nac_por_año.rename(columns={"Mbps (Media de bajada)": "Velocidad media", "Trimestre.1": "Periodo"}, inplace=True)
velocidad_nac_por_año['Velocidad media'] = velocidad_nac_por_año['Velocidad media'].apply(lambda x: x / 1000 if x > 350 else x)
ingresos_nac.rename(columns={"OTROS": "Otros"}, inplace=True)
ingresos_nac["Ingresos"] = ingresos_nac["Ingresos (miles de pesos)"] * 1000
ingresos_nac.drop(columns="Ingresos (miles de pesos)", inplace=True)
penetracion_nac["Accesos por cada 100 hogares"].replace(",", ".", regex=True, inplace=True)
penetracion_nac["Accesos por cada 100 hogares"] = penetracion_nac["Accesos por cada 100 hogares"].astype(float)
penetracion_nac["Accesos por cada 100 hab"].replace(",", ".", regex=True, inplace=True)
penetracion_nac["Accesos por cada 100 hab"] = penetracion_nac["Accesos por cada 100 hab"].astype(float)
accesos_nac_por_tecnologia = accesos_nac_por_tecnologia.merge(baf_nac[["Año", "Trimestre", "Banda ancha fija", "Dial up"]], on=["Año", "Trimestre"])
accesos_nac_por_tecnologia.rename(columns={"Dial up": "Dial Up", "Banda ancha fija": "BAF"}, inplace=True)
accesos_nac_por_velocidad.rename(columns={"OTROS": "Otros"}, inplace=True)

In [8]:
accesos_nac_por_tecnologia["ADSL"] = accesos_nac_por_tecnologia["ADSL"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Cablemodem"] = accesos_nac_por_tecnologia["Cablemodem"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Fibra óptica"] = accesos_nac_por_tecnologia["Fibra óptica"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Wireless"] = accesos_nac_por_tecnologia["Wireless"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Otros"] = accesos_nac_por_tecnologia["Otros"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Total"] = accesos_nac_por_tecnologia["Total"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["BAF"] = accesos_nac_por_tecnologia["BAF"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Dial Up"] = accesos_nac_por_tecnologia["Dial Up"].apply(lambda x: x * 1000 if x < 100 else x)
accesos_nac_por_tecnologia["Wireless"] = accesos_nac_por_tecnologia["Wireless"].apply(lambda x: x / 1000 if x > 50000 else x)
accesos_nac_por_tecnologia["Otros"] = accesos_nac_por_tecnologia["Otros"].apply(lambda x: x / 1000 if x > 50000 else x)

In [9]:
accesos_prov_por_tecnologia

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2022,4,Buenos Aires,326.0,3.0,1.0,130.0,72.0,5.0
1,2022,4,Capital Federal,139.0,1.0,139.0,5.0,32.0,2.0
2,2022,4,Catamarca,9.0,11.0,47.0,1.0,2.0,71.0
3,2022,4,Chaco,25.0,61.0,47.0,8.0,3.0,144.0
4,2022,4,Chubut,45.0,74.0,10.0,31.0,9.0,169.0
...,...,...,...,...,...,...,...,...,...
860,2014,1,Santa Fe,323.0,174.0,3.0,6.0,644.0,507.0
861,2014,1,Santiago Del Estero,33.0,4.0,19.0,915.0,34.0,37.0
862,2014,1,Tierra Del Fuego,22.0,3.0,648.0,1.0,3.0,28.0
863,2014,1,Tucumán,130.0,83.0,121.0,13.0,98.0,130.0


In [10]:
# informacion a nivel provincial
accesos_prov_por_tecnologia["Año"].replace("2019 *", "2019", inplace=True)
accesos_prov_por_tecnologia["Trimestre"].replace("1 *", "1", inplace=True)
accesos_prov_por_tecnologia["Trimestre"].replace("2 *", "2", inplace=True)
accesos_prov_por_tecnologia["Trimestre"].replace("3 *", "3", inplace=True)
accesos_prov_por_tecnologia.drop(labels=864, inplace=True)
accesos_prov_por_tecnologia["Año"] = accesos_prov_por_tecnologia["Año"].astype(int)
accesos_prov_por_tecnologia["Trimestre"] = accesos_prov_por_tecnologia["Trimestre"].astype(int)
#accesos_prov_por_tecnologia = accesos_prov_por_tecnologia.merge(baf_prov[["Año", "Trimestre", "Banda ancha fija", "Dial up"]], on=["Año", "Trimestre"])
#accesos_prov_por_tecnologia.rename(columns={"Dial up": "Dial Up", "Banda ancha fija": "BAF"}, inplace=True)
accesos_prov_por_velocidad.drop(labels=0, inplace=True)
accesos_prov_por_velocidad.fillna(0, inplace=True)
penetracion_prov.drop(columns=["Unnamed: 4", "Unnamed: 5", "Unnamed: 6"], inplace=True)
penetracion_prov["Accesos por cada 100 hogares"].replace(",", ".", regex=True, inplace=True)
penetracion_prov["Accesos por cada 100 hogares"] = penetracion_prov["Accesos por cada 100 hogares"].astype(float)

In [11]:
# informacion a nivel local
accesos_loc_por_tecnologia.rename(columns={"OTROS": "Otros", "CABLEMODEM": "Cablemodem", "FIBRA OPTICA": "Fibra óptica", "SATELITAL": "Satelital", "WIMAX": "Wimax", "WIRELESS": "Wireless", "DIAL UP": "Dial Up", "Total general": "Total"}, inplace=True)
accesos_loc_por_tecnologia.drop(columns="Unnamed: 13", inplace=True)
accesos_loc_por_tecnologia["ADSL"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["ADSL"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["ADSL"] = accesos_loc_por_tecnologia["ADSL"].astype(float)
accesos_loc_por_tecnologia["Cablemodem"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Cablemodem"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Cablemodem"] = accesos_loc_por_tecnologia["Cablemodem"].astype(float)
accesos_loc_por_tecnologia["Dial Up"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Dial Up"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Dial Up"] = accesos_loc_por_tecnologia["Dial Up"].astype(float)
accesos_loc_por_tecnologia["Fibra óptica"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Fibra óptica"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Fibra óptica"] = accesos_loc_por_tecnologia["Fibra óptica"].astype(float)
accesos_loc_por_tecnologia["Otros"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Otros"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Otros"] = accesos_loc_por_tecnologia["Otros"].astype(float)
accesos_loc_por_tecnologia["Satelital"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Satelital"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Satelitala"] = accesos_loc_por_tecnologia["Satelital"].astype(float)
accesos_loc_por_tecnologia["Wimax"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Wimax"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Wimax"] = accesos_loc_por_tecnologia["Wimax"].astype(float)
accesos_loc_por_tecnologia["Wireless"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Wireless"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Wireless"] = accesos_loc_por_tecnologia["Wireless"].astype(float)
accesos_loc_por_tecnologia["Total"].replace("- 0", "0", regex=True, inplace=True)
accesos_loc_por_tecnologia["Total"].replace(",", ".", regex=True, inplace=True)
accesos_loc_por_tecnologia["Total"] = accesos_loc_por_tecnologia["Total"].astype(float)
localidades_con_conectividad.rename(columns={"Cablemódem": "Cablemodem"}, inplace=True)
localidades_con_conectividad["ADSL"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["ADSL"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["Cablemodem"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["Cablemodem"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["Dial Up"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["Dial Up"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["Fibra óptica"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["Fibra óptica"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["4G"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["4G"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["3G"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["3G"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["Telefonía Fija"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["Telefonía Fija"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["Wireless"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["Wireless"].replace("--", False, regex=True, inplace=True)
localidades_con_conectividad["Satelital"].replace("SI", True, regex=True, inplace=True)
localidades_con_conectividad["Satelital"].replace("--", False, regex=True, inplace=True)

##### Exportar

In [12]:
# sig datasets
sig_provincias.to_csv("data/sig_provincias.csv")
sig_localidades.to_csv("data/sig_localidades.csv")

In [13]:
# informacion a nivel nacional
velocidad_nac_por_año.to_csv("data/velocidad_nac_por_año.csv")
accesos_nac_por_tecnologia.to_csv("data/accesos_nac_por_tecnologia.csv")
accesos_nac_por_velocidad.to_csv("data/accesos_nac_por_velocidad.csv")
ingresos_nac.to_csv("data/ingresos_nac.csv")
penetracion_nac.to_csv("data/penetracion_nac.csv")

In [14]:
# informacion a nivel provincial
accesos_prov_por_tecnologia.to_csv("data/accesos_prov_por_tecnologia.csv")
accesos_prov_por_velocidad.to_csv("data/accesos_prov_por_velocidad.csv")
penetracion_prov.to_csv("data/penetracion_prov.csv")

In [15]:
# informacion a nivel local
accesos_loc_por_tecnologia.to_csv("data/accesos_loc_por_tecnologia.csv")
accesos_loc_por_velocidad.to_csv("data/accesos_loc_por_velocidad.csv")
localidades_con_conectividad.to_csv("data/localidades_con_conectividad.csv")